In [1]:
import pytubefix 
from pytubefix import YouTube
import subprocess
import torch
import os
import ffmpeg
import pandas as pd
import re

pytubefix is used to work with youtube videos from within python
ffmpeg to convert between mp3 and wav, and other av file formats

In [2]:
device = torch.device('cpu') #or cuda if you have cuda

In [3]:
audio_mp3 = "../Dataset/mp3"
audio_wav = "../Dataset/wav"
subtitles_srt = "../Dataset/srt"
subtitles_txt = "../Dataset/txt"
os.makedirs(audio_mp3, exist_ok=True)
os.makedirs(audio_wav, exist_ok=True)
os.makedirs(subtitles_srt, exist_ok=True)
os.makedirs(subtitles_txt, exist_ok=True)


In [4]:
df = pd.read_csv("links.csv")
df['links']

0           https://www.youtube.com/watch?v=MpcQOSE3vM4
1      https://youtu.be/XCPZBD9lbVo?si=t4OpAlzVuUNUVFGU
2      https://youtu.be/iF1tOCEXLXY?si=LYHwhjce8rCX0_MR
3      https://youtu.be/ExoluwBk4Xc?si=cM9qSR4NFGfi7Tbw
4      https://youtu.be/zx_oMonGdK4?si=W6_uELtd45UQDRwh
5      https://youtu.be/uUnaBP-9zVI?si=6gJUFwo_8NbklJg0
6      https://youtu.be/52nI2IPyOu8?si=jECh7qI9pbUyd7zv
7      https://youtu.be/TMLyKcBtHuo?si=ghPQwesGR26DNfEg
8      https://youtu.be/xIEe0yK1VL4?si=W-ui1Jo_cn-0B9-a
9      https://youtu.be/dtGRmhZ6Cuo?si=qTl5Dc2oKhRZroex
10     https://youtu.be/ZOvRZ7UJMjk?si=nmt9d4Z0vIfxuzGE
Name: links, dtype: object

import re
video_url= "https://youtu.be/XCPZBD9lbVo?si=t4OpAlzVuUNUVFGU"
caption=YouTube(video_url).captions.get_by_language_code('a.en')
srt_content = caption.generate_srt_captions()

lines = srt_content.splitlines()
text = ''
for line in lines:
    if re.search('^[0-9]+$', line) is None and re.search('^[0-9]{2}:[0-9]{2}:[0-9]{2}', line) is None and re.search('^$', line) is None:
                text += ' ' + line.strip()
    text = text.lstrip()  

text

In [9]:
i=0
for video_url in df['links']:
    try:
        yt = YouTube(video_url)
        if yt.captions:
            caption = yt.captions.get_by_language_code('en.LUU0EuDKgKo')
            if caption:
                srt_content = caption.generate_srt_captions()
                lines = srt_content.splitlines()
                txt_content = ''
                for line in lines:
                    if re.search('^[0-9]+$', line) is None and re.search('^[0-9]{2}:[0-9]{2}:[0-9]{2}', line) is None and re.search('^$', line) is None:
                                txt_content += ' ' + line.strip()
                txt_content = txt_content.lstrip()
                
                # File paths
                srt_file_name = f"{subtitles_srt}/vid{i}.srt"
                txt_file_name = f"{subtitles_txt}/vid{i}.txt"
                
                # Save as .srt
                with open(srt_file_name, "w", encoding="utf-8") as f:
                    f.write(srt_content)
                print(f"Subtitles saved as SRT: {srt_file_name}")
                
                # Save as .txt
                with open(txt_file_name, "w", encoding="utf-8") as f:
                    f.write(txt_content)
                print(f"Subtitles saved as TXT: {txt_file_name}")
            else:
                caption = yt.captions.get_by_language_code('a.en')
                srt_content = caption.generate_srt_captions()
                lines = srt_content.splitlines()
                txt_content = ''
                for line in lines:
                    if re.search('^[0-9]+$', line) is None and re.search('^[0-9]{2}:[0-9]{2}:[0-9]{2}', line) is None and re.search('^$', line) is None:
                                txt_content += ' ' + line.strip()
                txt_content = txt_content.lstrip()
                srt_file_name = f"{subtitles_srt}/vid{i}.srt"
                txt_file_name = f"{subtitles_txt}/vid{i}.txt"
                
                # Save as .srt
                with open(srt_file_name, "w", encoding="utf-8") as f:
                    f.write(srt_content)
                print(f"Subtitles saved as SRT: {srt_file_name}")
                
                # Save as .txt
                with open(txt_file_name, "w", encoding="utf-8") as f:
                    f.write(txt_content)
                print(f"Subtitles saved as TXT: {txt_file_name}")

                adst = yt.streams.filter(only_audio=True, file_extension='mp4').order_by('abr').desc().first()
                adst.download(output_path=audio_mp3, filename=f"aud{i}.mp3")
                print(f"Audio downloaded as mp3: aud{i}.mp3")
                    
    except Exception as e:
        print(f"Failed to download subtitles for {video_url}: {e}")
    i+=1

C:\Users\spsin\AppData\Local\Temp\ipykernel_10448\1783760404.py:6: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('en.LUU0EuDKgKo')
C:\Users\spsin\AppData\Local\Temp\ipykernel_10448\1783760404.py:30: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('a.en')


Subtitles saved as SRT: ../Dataset/srt/vid0.srt
Subtitles saved as TXT: ../Dataset/txt/vid0.txt
Audio downloaded as mp3: aud0.mp3


C:\Users\spsin\AppData\Local\Temp\ipykernel_10448\1783760404.py:6: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('en.LUU0EuDKgKo')


Subtitles saved as SRT: ../Dataset/srt/vid1.srt
Subtitles saved as TXT: ../Dataset/txt/vid1.txt


C:\Users\spsin\AppData\Local\Temp\ipykernel_10448\1783760404.py:6: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('en.LUU0EuDKgKo')


...

Subtitles saved as SRT: ../Dataset/srt/vid8.srt
Subtitles saved as TXT: ../Dataset/txt/vid8.txt


C:\Users\spsin\AppData\Local\Temp\ipykernel_10448\1783760404.py:6: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('en.LUU0EuDKgKo')


Subtitles saved as SRT: ../Dataset/srt/vid9.srt
Subtitles saved as TXT: ../Dataset/txt/vid9.txt


C:\Users\spsin\AppData\Local\Temp\ipykernel_10448\1783760404.py:6: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  caption = yt.captions.get_by_language_code('en.LUU0EuDKgKo')


Subtitles saved as SRT: ../Dataset/srt/vid10.srt
Subtitles saved as TXT: ../Dataset/txt/vid10.txt


In [ ]:
audio_mp3_files = [f for f in os.listdir(audio_mp3) if f.endswith(".mp3")]
j=0
for i in audio_mp3_files:
    subprocess.run(['ffmpeg', '-i', os.path.join(audio_mp3, i), os.path.join(audio_wav, f"aud{j}.wav"), '-y'], check=True)
    print(f"Audio {j} converted to wav")
    j+=1